In [ ]:
# 순서도
# 0. 문장 입력받기
#   - 챗봇 응답이 맞는지 확인하기.
#   - 느낌표로 intent구분

# 1. intent분석
#   - input에서 느낌표 제거
#   - 찬의랑 합칠 때, intent별로 mode전환 할 수있는 코드 필요 

# 2. 유사도 분류 + 출력
#   - corpus 필요
#   - query문 필요(입력문장임)
#   - cosine similarity계산
#   - top5유사도 출력

# 4. 제목 클릭시 답변 출력해주기
#   - 
#   - 딕셔너리형으로 저장되어 있는 질문 답변 출력하기(반환이 list순서대로 나와서 일단을 딕셔너리 필요없음)


#논의가 필요한 사항, 느낌표로 intent구분 후 동작 상의해야함.

In [1]:
import numpy as np
import pandas as pd

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
def query_confirm(query_input): #챗봇을 호출하는 문장이 맞는지 확인(느낌표 check), 챗봇과 연동시 필요 없어질 수도 있음(파실할 떄 check가 가능하다면)
    global status_value
    if(query_input[0:1] == "!" or query_input[0:2] == "!!"):
        status_value= True
        return True
    else:
        status_value= False
        return False


In [4]:
#작동 상태관리 각 단계에서 이상이 없으면 True로 관리됨.
status_value = False

In [5]:
#코퍼스 불러오기

In [6]:
data = pd.read_csv("test_dummy_first.csv")

In [7]:
data

,id,날짜,질문내용,답변내용,질문자,답변자,채널명
0,1_0000001,2020.11.01,배고프다,밥먹어,신재욱,김창영,질문답변
1,1_0000002,2020.12.03,자바스크립트 공부방법,"생활코딩, 니꼴라스샘",탁민경,홍찬의,질문답변
2,1_0000003,2020.08.03,놀러가고 싶다,애버랜드가자,김창영,홍찬의,it지식공유
3,1_0000004,2019.04.15,넷플릭스 뭐가 재밌어,킹덤을 봐봐,홍찬의,신재욱,질문답변
4,2_0000001,2018.05.03,안드로이드 공부 방법 추천,생활코딩,신재욱,탁민경,질문답변
5,2_0000002,2017.03.13,아마존 인턴쉽,www.naver.com,서인하,서인하,it지식공유


In [8]:
corpus = data['질문내용'].values.tolist()

In [9]:
answer_corpus = data['답변내용'].values.tolist()

In [10]:
# corpus_question_id_dict = dict(zip(data['질문내용'].values.tolist(),data['id'].values.tolist()))
# corpus_id_answer_dict = dict(zip(data['id'].values.tolist(), data['답변내용'].values.tolist()))

In [11]:
# corpus_question_id_dict

In [12]:
# corpus_id_answer_dict

In [34]:
#0. 문장입력받기
query = input("느낌표로 시작하는 문장을 입력해주세요")

느낌표로 시작하는 문장을 입력해주세요!Android


In [35]:
def delete_exclamation_mark(input):
    if(input[0:1] == "!"):
        return input[1:]
    elif(input[0:2] == "!!"):
        return input[2:]

In [36]:
#1. intent 분석
query = delete_exclamation_mark(query)

In [37]:
print(query)

Android


In [38]:
#2. 유사도 분류 + 출력
from sentence_transformers import SentenceTransformer, util
import numpy as np

embedder = SentenceTransformer('distiluse-base-multilingual-cased')

top_k = 5
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)
query_embedding = embedder.encode(query, convert_to_tensor=True)
cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
cos_scores = cos_scores.cpu()


#We use np.argpartition, to only partially sort the top_k results
top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

print("\n\n======================\n\n")
print("Query:", query)
print("\n유사한 질문 리스트 Top 5")

for i, idx in enumerate(top_results[0:top_k]):
    print(str(i+1)+"번 "+corpus[idx].strip(), "(Score: %.4f)" % (cos_scores[idx]))
    








Query: Android

유사한 질문 리스트 Top 5
1번 아마존 인턴쉽 (Score: 0.3716)
2번 배고프다 (Score: 0.2471)
3번 넷플릭스 뭐가 재밌어 (Score: 0.2032)
4번 안드로이드 공부 방법 추천 (Score: 0.1848)
5번 자바스크립트 공부방법 (Score: 0.1729)


In [39]:
#2. 유사도 분류 + 출력
from sentence_transformers import SentenceTransformer, util
import numpy as np

embedder = SentenceTransformer('distiluse-base-multilingual-cased-v2')

top_k = 5
corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)
query_embedding = embedder.encode(query, convert_to_tensor=True)
cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
cos_scores = cos_scores.cpu()


#We use np.argpartition, to only partially sort the top_k results
top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

print("\n\n======================\n\n")
print("Query:", query)
print("\n유사한 질문 리스트 Top 5")

for i, idx in enumerate(top_results[0:top_k]):
    print(str(i+1)+"번 "+corpus[idx].strip(), "(Score: %.4f)" % (cos_scores[idx]))





Query: Android

유사한 질문 리스트 Top 5
1번 아마존 인턴쉽 (Score: 0.3716)
2번 배고프다 (Score: 0.2471)
3번 넷플릭스 뭐가 재밌어 (Score: 0.2032)
4번 안드로이드 공부 방법 추천 (Score: 0.1848)
5번 자바스크립트 공부방법 (Score: 0.1729)


In [26]:
find_answer = input("찾고 싶으신 질문은 어떤것 입니까?(1~5번 입력)")
print(answer_corpus[top_results[int(find_answer)-1]])

찾고 싶으신 질문은 어떤것 입니까?(1~5번 입력)1
킹덤을 봐봐
